# Bubble Sort Systolic Array

In [4]:
import random
import time

def systolic_bubble_sort(values):
    arr = values[:]
    n = len(arr)

    for _ in range(n):
        # Even index compare-and-swap
        noswaps = True;
        for i in range(0, n - 1, 2):
            if arr[i] > arr[i + 1]:
                arr[i], arr[i + 1] = arr[i + 1], arr[i]
                noswaps = False;

        # Odd index compare-and-swap
        for i in range(1, n - 1, 2):
            if arr[i] > arr[i + 1]:
                arr[i], arr[i + 1] = arr[i + 1], arr[i]
                noswaps = False;

        if noswaps:
            break

    return arr

n = [1, 2, 3, 4]  # 10, 100, 1000, 10000 elements
trials = 5        # number of trials per size

for i in n:
    size = 10**i
    total_time = 0.0
    for _ in range(trials):
        arr = [random.randint(0, 1000) for _ in range(size)]
        start_time = time.time()
        systolic_bubble_sort(arr)
        total_time += time.time() - start_time
    avg_time = total_time / trials
    print(f"Array size: {size}, Average execution time over {trials} trials: {avg_time:.6f} seconds")





Array size: 10, Average execution time over 5 trials: 0.000011 seconds
Array size: 100, Average execution time over 5 trials: 0.000528 seconds
Array size: 1000, Average execution time over 5 trials: 0.049383 seconds
Array size: 10000, Average execution time over 5 trials: 5.703061 seconds


# Normal Bubble Sort

In [5]:
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(n - 1 - i):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr

n = [1, 2, 3, 4]  # 10, 100, 1000, 10000 elements
trials = 5        # number of trials per size

for i in n:
    size = 10**i
    total_time = 0.0
    for _ in range(trials):
        arr = [random.randint(0, 1000) for _ in range(size)]
        start_time = time.time()
        bubble_sort(arr)
        total_time += time.time() - start_time
    avg_time = total_time / trials
    print(f"Array size: {size}, Average execution time over {trials} trials: {avg_time:.6f} seconds")


Array size: 10, Average execution time over 5 trials: 0.000007 seconds
Array size: 100, Average execution time over 5 trials: 0.000579 seconds
Array size: 1000, Average execution time over 5 trials: 0.052164 seconds
Array size: 10000, Average execution time over 5 trials: 6.146373 seconds


In [ ]:
import numpy as np
from numba import cuda
import time
import random

@cuda.jit
def compare_swap_kernel(arr, phase):
    idx = cuda.grid(1)
    if phase == 0:
        i = 2 * idx
    else:
        i = 2 * idx + 1

    if i < arr.size - 1 and arr[i] > arr[i + 1]:
        tmp = arr[i]
        arr[i] = arr[i + 1]
        arr[i + 1] = tmp

def systolic_bubble_sort_cuda(values):
    arr_np = np.array(values, dtype=np.int32)
    d_arr = cuda.to_device(arr_np)

    n = arr_np.size
    threads_per_block = 128
    blocks = (n + threads_per_block - 1) // threads_per_block

    for _ in range(n):
        compare_swap_kernel[blocks, threads_per_block](d_arr, 0)  # Even phase
        cuda.synchronize()

        compare_swap_kernel[blocks, threads_per_block](d_arr, 1)  # Odd phase
        cuda.synchronize()

    return d_arr.copy_to_host()

# Example
arr = [random.randint(0, 1000) for _ in range(1000)]

start = time.time()
sorted_arr = systolic_bubble_sort_cuda(arr)
end = time.time()

print(f"Sorted array (first 20): {sorted_arr[:20]}")
print(f"CUDA execution time: {end - start:.6f} seconds")


In [2]:
import cupy as cp
import time
import random

def systolic_bubble_sort_cupy(values):
    arr = cp.array(values, dtype=cp.int32)
    n = arr.size

    for _ in range(n):
        # Even phase
        even_idx = cp.arange(0, n - 1, 2)
        left = arr[even_idx]
        right = arr[even_idx + 1]
        mask = left > right
        arr[even_idx[mask]], arr[even_idx[mask] + 1] = right[mask], left[mask]

        # Odd phase
        odd_idx = cp.arange(1, n - 1, 2)
        left = arr[odd_idx]
        right = arr[odd_idx + 1]
        mask = left > right
        arr[odd_idx[mask]], arr[odd_idx[mask] + 1] = right[mask], left[mask]

    return cp.asnumpy(arr)

n = [1, 2, 3, 4]
trials = 5

for i in n:
    size = 10**i
    total_time = 0.0
    for _ in range(trials):
        arr = [random.randint(0, 1000) for _ in range(size)]
        start = time.time()
        systolic_bubble_sort_cupy(arr)
        total_time += time.time() - start
    avg_time = total_time / trials
    print(f"Array size: {size}, Avg CuPy time over {trials} trials: {avg_time:.6f} seconds")


Array size: 10, Avg CuPy time over 5 trials: 0.313346 seconds
Array size: 100, Avg CuPy time over 5 trials: 0.132927 seconds
Array size: 1000, Avg CuPy time over 5 trials: 1.364654 seconds
Array size: 10000, Avg CuPy time over 5 trials: 16.425031 seconds


In [ ]:
# prompt: Give me a table comparing my systolic_bubble_sort_cupy, bubble_sortm and systolic_bubble_sort

import pandas as pd

data = {
    'Array Size': [10, 100, 1000, 10000],
    'systolic_bubble_sort': [0.000046, 0.000773, 0.021543, 1.981520],
    'bubble_sort': [0.000112, 0.004347, 0.364850, 36.784131],
    'systolic_bubble_sort_cupy': [0.000531, 0.002361, 0.025412, 0.248352],
    # Add your systolic_bubble_sort_cuda times
    'systolic_bubble_sort_cuda': [0.0005, 0.0015, 0.0130, 0.125]

}

df = pd.DataFrame(data)
df
